In [2]:
import rasterio
from rasterio.crs import CRS
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject
from rasterio.mask import mask
from rasterio.features import shapes
from shapely.geometry import shape, mapping
from shapely.ops import unary_union
import numpy as np

In [4]:
def process_nc_to_tif(nc_input_path, temp_tif_output_path, ref_raster_path, aligned_tif_path, final_output_path):
    # Open the NetCDF file
    with rasterio.open(nc_input_path) as nc_file:
        meta = nc_file.meta.copy()
        meta.update({
            'driver': 'GTiff',
            'height': nc_file.height,
            'width': nc_file.width,
            'crs': CRS.from_epsg(4326),
            'transform': rasterio.transform.from_origin(nc_file.bounds.left, nc_file.bounds.top, nc_file.res[0], nc_file.res[1])
        })

        # Read data
        data = nc_file.read(1)

        # Write data to GeoTIFF
        with rasterio.open(temp_tif_output_path, 'w', **meta) as dst:
            dst.write(data, 1)

    # Align and reproject the tif
    with rasterio.open(temp_tif_output_path) as src, rasterio.open(ref_raster_path) as ref:
        transform, width, height = calculate_default_transform(
            src.crs, ref.crs, ref.width, ref.height, *ref.bounds)
        meta = src.meta.copy()
        meta.update({
            'crs': ref.crs,
            'transform': transform,
            'width': width,
            'height': height,
            'driver': 'GTiff'
        })
        
        with rasterio.open(aligned_tif_path, 'w', **meta) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=ref.crs,
                    resampling=Resampling.nearest)

    # Clip the raster using the reference raster as a mask
    polygon = create_mask_polygon(ref_raster_path)
    if polygon:
        clip_raster_with_polygon(aligned_tif_path, final_output_path, polygon)

def create_mask_polygon(raster_path):
    with rasterio.open(raster_path) as src:
        image = src.read(1)
        mask = image > 0
        transform = src.transform
        shape_gen = shapes(image, mask=mask, transform=transform)
        polygons = [shape(geom) for geom, value in shape_gen if value]
        if polygons:
            return unary_union(polygons)
    return None

def clip_raster_with_polygon(raster_path, output_path, polygon):
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, [mapping(polygon)], crop=True)
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform
        })

        with rasterio.open(output_path, 'w', **out_meta) as out_raster:
            out_raster.write(out_image)

# Define the data types and process each
data_types = ['max', 'min', 'range']
base_path = 'data/aggregated_over_years_model_input/sm_{}.nc'
ref_raster_path = 'data/dead/deadwood.tif'

for data_type in data_types:
    nc_input = base_path.format(data_type)
    temp_tif_output = 'data/temp/sm_{}_temp.tif'.format(data_type)
    aligned_tif_output = 'data/temp/sm_{}_aligned.tif'.format(data_type)
    final_output_path = 'data/aggregated_over_years_model_input/sm_{}_pro.tif'.format(data_type)
    process_nc_to_tif(nc_input, temp_tif_output, ref_raster_path, aligned_tif_output, final_output_path)
